In [8]:
import geopandas as gpd
import requests
import json
import uuid
import os
from dotenv import load_dotenv, find_dotenv
from datetime import datetime

In [9]:
load_dotenv(find_dotenv())
archive_path = os.getenv('ZIP_PATH')
file_path = os.getenv('SHP_PATH')
disk_token = os.getenv('YD_TOKEN')
local_file_path = os.getenv("LOCAL_DATA")
remote_file_path = os.getenv("REMOTE_DATA")
disk_base_folder = os.getenv("FOLDER_PATH")
disk_base_url = os.getenv("BASE_URL")

In [10]:
def read_and_remove_archive(archive_path, file_path):
    try:
        gdf = gpd.read_file(f"zip://{archive_path}!{file_path}", encoding="utf-8")

        if os.path.exists(archive_path):
            os.remove(archive_path)
            print("Shapefile обработан и удалён.")
        else:
            print("Shapefile для удаления не обнаружен.")

        return gdf
    except Exception as err:
        print(f"Ошибка при чтении Shapefile: {err}")
        return None

In [11]:
gdf = read_and_remove_archive(archive_path, file_path)

if gdf is not None:
    print("Данные успешно загружены")

Shapefile обработан и удалён.
Данные успешно загружены


In [12]:
def extract_geometry(geometry):
    geom_type = geometry.geom_type

    if geom_type == 'Polygon':
        return [
            [list(coord) for coord in geometry.exterior.coords],
            *[[list(coord) for coord in interior.coords] for interior in geometry.interiors]
        ]
    elif geom_type == 'MultiPolygon':
        return [
            [
                [list(coord) for coord in poly.exterior.coords],
                *[[list(coord) for coord in interior.coords] for interior in poly.interiors]
            ]
            for poly in geometry.geoms
        ]
    elif geom_type == 'LineString':
        return [list(coord) for coord in geometry.coords]
    elif geom_type == 'MultiLineString':
        return [list(list(coords) for coords in line.coords) for line in geometry.geoms]
    elif geom_type == 'Point':
        return list(geometry.coords[0])
    elif geom_type == 'MultiPoint':
        return [list(point.coords[0]) for point in geometry.geoms]
    elif geom_type == 'GeometryCollection':
        result = []
        for geom in geometry.geoms:
            try:
                result.append(extract_geometry(geom))
            except ValueError:
                continue
        return result
    else:
        raise ValueError(f"Неподдерживаемый тип геометрии: {geom_type}")

In [13]:
output = {
    "paths": {},
    "points": {}
}

In [14]:
for _, row in gdf.iterrows():
    try:
        geom = row.geometry
        geom_type = geom.geom_type

        if geom_type in ['Polygon', 'MultiPolygon']:
            coords = extract_geometry(geom)
            if isinstance(coords[0], list):
                for poly_data in coords:
                    poly_uuid = str(uuid.uuid4())
                    output["paths"][poly_uuid] = poly_data
            else:
                poly_uuid = str(uuid.uuid4())
                output["paths"][poly_uuid] = coords

        elif geom_type in ['LineString', 'MultiLineString', 'GeometryCollection']:
            coords = extract_geometry(geom)
            if isinstance(coords[0], list):
                for path_coords in coords:
                    poly_uuid = str(uuid.uuid4())
                    output["paths"][poly_uuid] = [path_coords]
            else:
                poly_uuid = str(uuid.uuid4())
                output["paths"][poly_uuid] = [coords]

        elif geom_type in ['Point', 'MultiPoint']:
            coords = extract_geometry(geom)
            if isinstance(coords[0], list):
                for point_coords in coords:
                    point_uuid = str(uuid.uuid4())
                    output["points"][point_uuid] = point_coords
            else:
                point_uuid = str(uuid.uuid4())
                output["points"][point_uuid] = coords

    except ValueError as e:
        print(f"Пропущена геометрия: {e}")
    except Exception as e:
        print(f"Ошибка при обработке строки: {e}")

In [15]:
with open("index.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=4)

In [16]:
current_date_folder = datetime.now().strftime("%Y-%m-%d")

In [17]:
def check_and_create_root_folder(disk_base_url, disk_base_folder, disk_token):
    headers = {"Authorization": f"OAuth {disk_token}"}

    response = requests.get(f'{disk_base_url}?path={disk_base_folder}', headers=headers)

    if response.status_code == 200:
        print("Папка уже существует.")
        return True

    elif response.status_code == 404:
        response = requests.put(f'{disk_base_url}?path={disk_base_folder}', headers=headers)

        if response.status_code == 201:
            print("Папка успешно создана.")
            return True
        else:
            print(f"Ошибка при создании папки: {response.json()}")
            return False
    else:
        print(f"Ошибка при проверке папки: {response.json()}")
        return False

In [18]:
check_and_create_root_folder(disk_base_url, disk_base_folder, disk_token)

Папка уже существует.


True

In [19]:
def check_or_create_folder(disk_token, disk_base_folder, current_date_folder):
    headers = {"Authorization": f"OAuth {disk_token}"}
    full_path = f"{disk_base_folder}/{current_date_folder}"

    response = requests.get(f"{disk_base_url}?path={full_path}", headers=headers)

    if response.status_code == 200:
        return True

    elif response.status_code == 404:
        response = requests.put(f"{disk_base_url}?path={full_path}", headers=headers)

        if response.status_code == 201:
            print(f"На диске создана папка {full_path}")
            return True

    print(f"Ошибка при работе с папкой: {response.json()}")
    return False

In [20]:
check_or_create_folder(disk_token, disk_base_folder, current_date_folder)

True

In [21]:
def upload_to_ydisk(local_file_path, disk_base_folder, disk_token):
    headers = {"Authorization": f"OAuth {disk_token}"}
    file_name = os.path.basename(local_file_path)

    params = {
        "path": f"{disk_base_folder}/{current_date_folder}/{file_name}",
        "overwrite": "true"
    }

    response = requests.get(f"{disk_base_url}/upload", headers=headers, params=params)

    if response.status_code != 200:
        print("Ошибка при получении ссылки:", response.json())
        return False

    upload_href = response.json()["href"]

    with open(local_file_path, "rb") as file:
        upload_response = requests.put(upload_href, data=file)

    if upload_response.status_code == 201:
        print("Геометрия загружена в блокнот картографа!")

        try:
            os.remove(local_file_path)
        except OSError as e:
            print(f"Ошибка при удалении файла: {e}")
        return True

    else:
        print("Ошибка при загрузке в блокнот картографа:", upload_response.text)
        return False

In [22]:
upload_to_ydisk(local_file_path, disk_base_folder, disk_token)

Геометрия загружена в блокнот картографа!


True